# Création d'un dictionnaire qui contient un tableau pour chaque ligne de métro

## Chaque tableau affiche les stations de la ligne, leur fréquentaiton annuelle, leur fréquentation hebdomadaire moyenne sur la période 24/07/2022-11/08/2022 (période des JO mais en 2022)

In [2]:
import pandas as pd
import numpy as np
import re
import difflib

Copie du programme Nettoyage données affluence par jour + nombre de validation par jour par station

In [3]:
%config NotebookApp.iopub_data_rate_limit=100000000000.0
url = '/workspaces/metro-JO/Données importées/2022_S2_PROFIL_FER.csv'
données_validation = pd.read_csv(url, delimiter=';')

# Convertir la colonne 'pourc_validations' en float (remplacer la virgule par un point pour séparer les décimales)
données_validation['pourc_validations'] = données_validation['pourc_validations'].str.replace(',', '.').astype(float)

# Grouper par 'LIBELLE_ARRET' et 'TRNC_HORR_60', puis calculer la somme des pourc_validations pour chaque groupe
tableau_somme_pour_validations = données_validation.groupby(['LIBELLE_ARRET', 'TRNC_HORR_60'])['pourc_validations'].sum().reset_index()


# Mapping des tranches horaires à des heures spécifiques
tranche_horaire_to_hour = {
    '0H-1H': '00:00-01:00',
    '1H-2H': '01:00-02:00',
    '2H-3H': '02:00-03:00',
    '3H-4H': '03:00-04:00',
    '4H-5H': '04:00-05:00',
    '5H-6H': '05:00-06:00',
    '6H-7H': '06:00-07:00',
    '7H-8H': '07:00-08:00',
    '8H-9H': '08:00-09:00',
    '9H-10H': '09:00-10:00',
    '10H-11H': '10:00-11:00',
    '11H-12H': '11:00-12:00',
    '12H-13H': '12:00-13:00',
    '13H-14H': '13:00-14:00',
    '14H-15H': '14:00-15:00',
    '15H-16H': '15:00-16:00',
    '16H-17H': '16:00-17:00',
    '17H-18H': '17:00-18:00',
    '18H-19H': '18:00-19:00',
    '19H-20H': '19:00-20:00',
    '20H-21H': '20:00-21:00',
    '21H-22H': '21:00-22:00',
    '22H-23H': '22:00-23:00',
    '23H-24H': '23:00-00:00'
}

# Appliquer la conversion des tranches horaires au format heure
tableau_somme_pour_validations['TRNC_HORR_60'] = tableau_somme_pour_validations['TRNC_HORR_60'].map(tranche_horaire_to_hour)

# Regroupement des données en cas de doublons pour une paire 'LIBELLE_ARRET'-'TRNC_HORR_60'
grouped_data = données_validation.groupby(['LIBELLE_ARRET', 'TRNC_HORR_60'])['pourc_validations'].sum().reset_index()

# Pivotement des données
pivot_table = grouped_data.pivot(index='LIBELLE_ARRET', columns='TRNC_HORR_60', values='pourc_validations').reset_index()

# Renommage des colonnes
pivot_table.columns.name = None  # Supprimer le nom de la colonne index
pivot_table = pivot_table.rename(columns=lambda x: f"{x}" if x != 'LIBELLE_ARRET' else x)

# Sélectionner les colonnes correspondant aux tranches horaires
horaire_columns = pivot_table.columns[1:]  # Exclure la colonne 'LIBELLE_ARRET'

# Fonction pour extraire l'heure de début de chaque tranche horaire
def extract_start_hour(column):
    match = re.match(r'(\d+)H-(\d+)H', column)
    if match:
        return int(match.group(1))
    return 0

# Trier les colonnes dans l'ordre croissant des heures de début des tranches
sorted_columns = sorted(horaire_columns, key=extract_start_hour)

# Réorganiser le tableau pivoté avec les colonnes dans l'ordre chronologique
new_column_order = ['LIBELLE_ARRET'] + sorted_columns
pivot_table = pivot_table[new_column_order]

# Supprimer la colonne 'ND' du DataFrame pivot_table_ordered
pivot_table.drop(columns=['ND'], inplace=True, errors='ignore')

pivot_table['LIBELLE_ARRET'] = pivot_table['LIBELLE_ARRET'].str.strip()  # Supprime les espaces en début et fin
pivot_table['LIBELLE_ARRET'] = pivot_table['LIBELLE_ARRET'].str.lower()  # Met tout en minuscules

# Créer un dictionnaire de correspondance pour renommer les colonnes
correspondance_heures = {
    '0H-1H': '00:00-01:00', '1H-2H': '01:00-02:00', '2H-3H': '02:00-03:00', '3H-4H': '03:00-04:00',
    '4H-5H': '04:00-05:00', '5H-6H': '05:00-06:00', '6H-7H': '06:00-07:00', '7H-8H': '07:00-08:00',
    '8H-9H': '08:00-09:00', '9H-10H': '09:00-10:00', '10H-11H': '10:00-11:00', '11H-12H': '11:00-12:00',
    '12H-13H': '12:00-13:00', '13H-14H': '13:00-14:00', '14H-15H': '14:00-15:00', '15H-16H': '15:00-16:00',
    '16H-17H': '16:00-17:00', '17H-18H': '17:00-18:00', '18H-19H': '18:00-19:00', '19H-20H': '19:00-20:00',
    '20H-21H': '20:00-21:00', '21H-22H': '21:00-22:00', '22H-23H': '22:00-23:00', '23H-0H': '23:00-00:00'
}

# Renommer les colonnes en utilisant le dictionnaire de correspondance
pivot_table.rename(columns=correspondance_heures, inplace=True)

/tmp/ipykernel_30756/1420814720.py:3: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  données_validation = pd.read_csv(url, delimiter=';')


Importation ableau avec station, ligne, correspondances... (Données disponibles sur le site https://data.ratp.fr)

In [5]:
%pip install openpyxl
tableau_fréquentation = pd.read_excel('https://data.ratp.fr/api/explore/v2.1/catalog/datasets/trafic-annuel-entrant-par-station-du-reseau-ferre-2021/exports/xlsx?lang=fr&timezone=Europe%2FBerlin&use_labels=true')

tableau_fréquentation = tableau_fréquentation[tableau_fréquentation['Réseau'] == 'Métro']
tableau_frequentation = tableau_fréquentation.drop_duplicates(subset='Station')


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Nettoyage du tableau

In [24]:
colonnes_a_supprimer = ['Réseau', 'Ville', 'Arrondissement pour Paris']

tableau_fréquentation = tableau_fréquentation.drop(colonnes_a_supprimer, axis=1)

# Liste des colonnes à modifier
colonnes_correspondance = ['Correspondance_1', 'Correspondance_2', 'Correspondance_3', 'Correspondance_4', 'Correspondance_5']

# Convertir les colonnes spécifiées en type chaîne (si elles ne le sont pas déjà)
tableau_fréquentation[colonnes_correspondance] = tableau_fréquentation[colonnes_correspondance].astype(str)

# Supprimer les lettres 'A' et 'B' des colonnes spécifiées dans tableau_fréquentation
for colonne in colonnes_correspondance:
    tableau_fréquentation[colonne] = tableau_fréquentation[colonne].str.replace('A', '').str.replace('B', '')

### PROBLÈME À RÉGLER : CREER UNE COLONNE QUI COMPTE LE NOMBRE DE LIGNES (ELLE S'APPELLE NB_CORRESPONDANCE) : CI DESSOUS J'AI ESSAYÉ MAIS CA AFFICHE QUE DES 5 ###

In [26]:
# Liste des colonnes à analyser
colonnes_correspondance = ['Correspondance_1', 'Correspondance_2', 'Correspondance_3', 'Correspondance_4', 'Correspondance_5']

# Calcul du nombre de cases non vides pour chaque ligne
tableau_fréquentation['nb_correspondance'] = tableau_fréquentation[colonnes_correspondance].count(axis=1)

display(tableau_fréquentation)

,Rang,Station,Trafic,Correspondance_1,Correspondance_2,Correspondance_3,Correspondance_4,Correspondance_5,nb_correspondance
0,3,GARE DE LYON,28640475,1,14,,nan,nan,5
1,12,BELLEVILLE,7314438,2,11,nan,nan,nan,5
2,32,OPERA,5193831,3,7,8,nan,nan,5
3,58,SAINT-MANDE-TOURELLE,3944640,1,nan,nan,nan,nan,5
4,59,HOCHE,3928404,5,nan,nan,nan,nan,5
...,...,...,...,...,...,...,...,...,...
350,272,SULLY-MORLAND,1124169,7,nan,nan,nan,nan,5
351,275,EUROPE,1106459,3,nan,nan,nan,nan,5
352,283,CAMPO-FORMIO,1016150,5,nan,nan,nan,nan,5
353,298,SAINT-FARGEAU,472258,3bis,nan,nan,nan,nan,5


In [27]:
# Diviser les valeurs de la colonne "Trafic" par les valeurs de la colonne "nb_correspondance"
tableau_fréquentation['Trafic'] = tableau_fréquentation['Trafic'] / tableau_fréquentation['nb_correspondance']
print(tableau_fréquentation)

     Rang               Station     Trafic Correspondance_1 Correspondance_2  \
0       3          GARE DE LYON  5728095.0                1               14   
1      12            BELLEVILLE  1462887.6                2               11   
2      32                 OPERA  1038766.2                3                7   
3      58  SAINT-MANDE-TOURELLE   788928.0                1              nan   
4      59                 HOCHE   785680.8                5              nan   
..    ...                   ...        ...              ...              ...   
350   272         SULLY-MORLAND   224833.8                7              nan   
351   275                EUROPE   221291.8                3              nan   
352   283          CAMPO-FORMIO   203230.0                5              nan   
353   298         SAINT-FARGEAU    94451.6             3bis              nan   
354   300       PORTE D'AUTEUIL    75149.6               10              nan   

    Correspondance_3 Correspondance_4 C

Création d'un tableau avec la moyenne sur la période des JO en 2022 du nombre de validation hebdomadaire à chaque station

In [28]:
#Nettoyage de données sur le nombre de validations: on obtient le nombre de validation moyen par jour sur 1 mois précis

import pandas as pd
%config NotebookApp.iopub_data_rate_limit=100000000000.0
url2 = '/workspaces/metro-JO/Données importées/2022_S2_NB_FER.csv'
données_validation2= pd.read_csv(url2)
données_validation2 = pd.read_csv(url2, delimiter=';')

# Exemple avec gestion de formats flexibles
données_validation2['JOUR'] = pd.to_datetime(données_validation2['JOUR'], errors='coerce', format="%d/%m/%Y")
donnees_mai = données_validation2.loc[(données_validation2['JOUR'].dt.month == 5)]
donnees_sum = donnees_mai.groupby('LIBELLE_ARRET')['NB_VALD'].mean().reset_index()

# Conversion de la colonne 'JOUR' en datetime
données_validation2['JOUR'] = pd.to_datetime(données_validation2['JOUR'], format='%d/%m/%Y')

# Filtrer les données pour les dates spécifiques
donnees_filtrees = données_validation2[(données_validation2['JOUR'] >= '2022-07-24') & (données_validation2['JOUR'] <= '2022-08-11')]

# Grouper par 'JOUR', 'LIBELLE_ARRET' et 'CATEGORIE_TITRE' et calculer la somme des 'NB_VALD'
donnees_aggreg = donnees_filtrees.groupby(['JOUR', 'LIBELLE_ARRET', 'CATEGORIE_TITRE'])['NB_VALD'].sum().reset_index()

# Conversion de la colonne 'JOUR' en datetime si ce n'est pas déjà fait
données_validation2['JOUR'] = pd.to_datetime(données_validation2['JOUR'], format='%d/%m/%Y')

# Filtrage des dates spécifiques
donnees_filtrees = données_validation2[(données_validation2['JOUR'] >= '2022-07-24') & (données_validation2['JOUR'] <= '2022-08-11')]

# Somme de NB_VALD par JOUR et par LIBELLE_ARRET
somme_par_jour_arret = donnees_filtrees.groupby(['JOUR', 'LIBELLE_ARRET'])['NB_VALD'].sum().reset_index()

moyenne_par_arret = somme_par_jour_arret.groupby('LIBELLE_ARRET')['NB_VALD'].mean().reset_index()

# Normalisation des libellés
moyenne_par_arret['LIBELLE_ARRET'] = moyenne_par_arret['LIBELLE_ARRET'].str.lower().str.strip()

# Somme des valeurs de NB_VALD pour les libellés normalisés
moyenne_par_arret = moyenne_par_arret.groupby('LIBELLE_ARRET')['NB_VALD'].sum().reset_index()

# Suppression des doublons
moyenne_par_arret = moyenne_par_arret.drop_duplicates(subset=['LIBELLE_ARRET'])

display(moyenne_par_arret)

,LIBELLE_ARRET,NB_VALD
0,abbesses,2277.315789
1,ablon,1101.105263
2,acheres grand,51.388889
3,acheres ville,3012.052632
4,aeroport cdg1,5050.947368
...,...,...
733,vosves,14.894737
734,vulaines sur se,42.894737
735,wagram,3268.157895
736,yerres,2743.000000


Création d'un dictionnaire contenant les fréquentations de chaque station de chaque ligne 

In [29]:
numéro = [1, 2, 3, '3bis', 4, 5, 6, 7, '7bis', 8, 9, 10, 11, 12, 13, 14]

# Création des tableaux pour chaque élément de numéro
tableaux = {}

colonnes_correspondance = ['Correspondance_1', 'Correspondance_2', 'Correspondance_3', 'Correspondance_4', 'Correspondance_5']

for k in numéro:
    # Convertir k en string car il peut contenir des valeurs non numériques
    k_str = str(k)
    
    # Filtrer les lignes où la valeur k est présente dans les colonnes spécifiées
    filtered_rows = tableau_fréquentation[tableau_fréquentation[colonnes_correspondance].apply(lambda x: k_str in x.values, axis=1)]
    
    # Stocker les lignes filtrées dans un tableau nommé "ligne numéro[k]"
    tableaux[f'ligne numéro[{k_str}]'] = filtered_rows

# Parcourir chaque tableau dans le dictionnaire et supprimer les colonnes spécifiées
for tableau_nom, tableau in tableaux.items():
    tableaux[tableau_nom] = tableau.drop(columns=colonnes_correspondance)
    tableau = tableau.drop_duplicates(subset='Station')


### PROBLÈME À RÉGLER AVANT L'ETAPE SUIVANTE : ON A DES STATIONS EN DOUBLON -> DU COUP LA FREQUENTATION TOTALE PAR LIGNE ET LES PART DU TRAFIC DE LA STATION PAR RAPPORT À TOUTE LA LIGNE SONT MAUVAISES ###

In [30]:

# Parcourir chaque tableau dans le dictionnaire
for tableau_nom, tableau in tableaux.items():
    # Calculer la fréquentation totale pour chaque tableau
    fréquentation_totale_ligne = tableau['Trafic'].sum()

    # Ajouter la variable fréquentation_totale à chaque tableau
    tableaux[tableau_nom]['fréquentation_totale_ligne'] = fréquentation_totale_ligne
    tableau['Part du trafic de la station par rapport à toute la ligne'] = tableau['Trafic'] / tableau['fréquentation_totale_ligne']

# Supposons que la clé pour la correspondance soit 'LIBELLE_ARRET' dans le tableau moyenne_par_arret
cle_correspondance = 'LIBELLE_ARRET'

# Nettoyage des espaces et uniformisation de la casse
moyenne_par_arret['LIBELLE_ARRET'] = moyenne_par_arret['LIBELLE_ARRET'].str.strip().str.lower()
for tableau_nom, tableau in tableaux.items():
    tableau['Station'] = tableau['Station'].str.strip().str.lower()

    # Fusion en utilisant les noms de stations nettoyés
    merged_table = pd.merge(tableau, moyenne_par_arret, left_on='Station', right_on='LIBELLE_ARRET', how='left')
    
    # Renommer la colonne NB_VALD de moyenne_par_arret
    merged_table = merged_table.rename(columns={'NB_VALD': 'fréquentation moyenne hors JO'})
    
    tableaux[tableau_nom] = merged_table

In [31]:
#compléter avec les valeurs manquantes

# Méthode pour trouver la correspondance la plus proche
def find_closest_match(target, candidates):
    closest_match = difflib.get_close_matches(target, candidates, n=1, cutoff=0.4)  # Ajustez le seuil selon vos besoins
    return closest_match[0] if closest_match else None

for k in numéro:
    k_str = str(k)
    # Accéder aux tableaux spécifiques
    current_table = tableaux[f'ligne numéro[{k_str}]']
    
    # Filtrer les lignes avec des valeurs manquantes dans la colonne 'LIBELLE_ARRET'
    missing_data = current_table[current_table['LIBELLE_ARRET'].isnull()]
    
    # Remplir les valeurs manquantes
    for index, row in missing_data.iterrows():
        closest_match = find_closest_match(row['Station'], moyenne_par_arret['LIBELLE_ARRET'])
        
        if closest_match:
            tableaux[f'ligne numéro[{k_str}]'].at[index, 'LIBELLE_ARRET'] = closest_match



In [32]:
for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Fusionner avec le tableau moyenne_par_arret pour obtenir les valeurs NB_VALD associées
    merged_with_nb_vald = pd.merge(current_table, moyenne_par_arret[['LIBELLE_ARRET', 'NB_VALD']],
                                   left_on='LIBELLE_ARRET', right_on='LIBELLE_ARRET', how='left')
    
    # Renommer la colonne NB_VALD pour plus de clarté
    merged_with_nb_vald = merged_with_nb_vald.rename(columns={'NB_VALD': 'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station'})

    # Mettre à jour le tableau dans le dictionnaire
    tableaux[f'ligne numéro[{k_str}]'] = merged_with_nb_vald

for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Fusionner les données avec le tableau contenant les nouvelles colonnes
    merged_table = pd.merge(current_table, pivot_table, on='LIBELLE_ARRET', how='left')

    # Mettre à jour le tableau dans le dictionnaire avec les nouvelles colonnes fusionnées
    tableaux[f'ligne numéro[{k_str}]'] = merged_table

In [33]:
for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Supprimer les colonnes 'fréquentation moyenne hors JO' et 'LIBELLE_ARRET' si elles existent
    columns_to_drop = ['fréquentation moyenne hors JO', 'LIBELLE_ARRET']
    current_table = current_table.drop(columns=columns_to_drop, errors='ignore')

    # Mettre à jour le tableau dans le dictionnaire
    tableaux[f'ligne numéro[{k_str}]'] = current_table

In [34]:
for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Vérifier si les colonnes existent avant de créer la nouvelle colonne
    if 'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station' in current_table.columns and 'nb_correspondance' in current_table.columns:
        # Calcul de la nouvelle colonne
        current_table['moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne'] = current_table['moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station'] / current_table['nb_correspondance']

        # Mettre à jour le tableau dans le dictionnaire
        tableaux[f'ligne numéro[{k_str}]'] = current_table

for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Vérifier si les colonnes existent avant de créer la nouvelle colonne
    if 'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station' in current_table.columns and 'nb_correspondance' in current_table.columns:
        # Calcul de la nouvelle colonne
        current_table['moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction'] = current_table['moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station'] / current_table['nb_correspondance'] / 2

        # Mettre à jour le tableau dans le dictionnaire
        tableaux[f'ligne numéro[{k_str}]'] = current_table

for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Vérifier si la colonne existe avant de créer la nouvelle colonne
    if 'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne' in current_table.columns:
        # Calculer la somme de la colonne spécifique
        current_table['nombre_validations_totales_pour_la_ligne'] = current_table['moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne'].sum()

        # Mettre à jour le tableau dans le dictionnaire
        tableaux[f'ligne numéro[{k_str}]'] = current_table
        # Afficher le tableau mis à jour
        print(f"Tableau 'ligne numéro[{k_str}]' avec la nouvelle colonne 'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction' :")
        display(tableaux[f'ligne numéro[{k_str}]'])
        tableaux[f'ligne numéro[{k_str}]'].to_csv(f'ligne_numero_{k_str}.csv', index=False) 


Tableau 'ligne numéro[1]' avec la nouvelle colonne 'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction' :


,Rang,Station,Trafic,nb_correspondance,fréquentation_totale_ligne,Part du trafic de la station par rapport à toute la ligne,moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station,00:00-01:00,01:00-02:00,02:00-03:00,...,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nombre_validations_totales_pour_la_ligne
0,3,gare de lyon,5728095.0,5,27894284.0,0.205350,66037.947368,2.16,0.01,0.00,...,57.99,60.21,38.73,27.60,17.40,10.24,4.80,13207.589474,6603.794737,138177.887449
1,3,gare de lyon,5728095.0,5,27894284.0,0.205350,66037.947368,18.38,4.31,0.30,...,84.89,89.26,74.77,56.04,41.78,38.76,34.89,13207.589474,6603.794737,138177.887449
2,58,saint-mande-tourelle,788928.0,5,27894284.0,0.028283,8727.684211,3.59,1.06,0.10,...,44.98,47.84,34.26,19.04,10.38,8.05,5.99,1745.536842,872.768421,138177.887449
3,192,louvre,373922.4,5,27894284.0,0.013405,4374.473684,14.09,4.62,0.10,...,59.32,63.31,49.76,28.05,19.06,19.83,20.34,874.894737,437.447368,138177.887449
4,25,porte de vincennes,1089320.4,5,27894284.0,0.039052,12151.052632,3.90,1.27,0.10,...,40.51,38.45,30.06,18.82,12.32,9.06,6.66,2430.210526,1215.105263,138177.887449
5,51,porte maillot,827660.2,5,27894284.0,0.029671,7384.429150,2.64,0.07,0.02,...,68.87,69.45,51.21,31.13,17.93,11.92,7.54,1476.885830,738.442915,138177.887449
6,51,porte maillot,827660.2,5,27894284.0,0.029671,7384.429150,10.76,3.99,0.11,...,44.22,44.75,35.65,22.26,15.20,16.55,18.25,1476.885830,738.442915,138177.887449
7,71,chateau de vincennes,723547.6,5,27894284.0,0.025939,12151.052632,3.90,1.27,0.10,...,40.51,38.45,30.06,18.82,12.32,9.06,6.66,2430.210526,1215.105263,138177.887449
8,10,chatelet,1670158.8,5,27894284.0,0.059875,87779.526316,30.17,10.60,0.27,...,114.31,122.78,105.60,70.85,51.84,47.85,44.60,17555.905263,8777.952632,138177.887449
9,41,pont de neuilly,961900.6,5,27894284.0,0.034484,10573.157895,3.28,1.22,0.13,...,46.88,51.98,39.18,21.86,11.94,9.25,6.68,2114.631579,1057.315789,138177.887449


Tableau 'ligne numéro[2]' avec la nouvelle colonne 'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction' :


,Rang,Station,Trafic,nb_correspondance,fréquentation_totale_ligne,Part du trafic de la station par rapport à toute la ligne,moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station,00:00-01:00,01:00-02:00,02:00-03:00,...,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nombre_validations_totales_pour_la_ligne
0,12,belleville,1462887.6,5,16877281.0,0.086678,16611.631579,7.73,3.56,0.12,...,41.62,37.49,32.01,21.30,14.72,13.35,11.97,3322.326316,1661.163158,70635.936842
1,104,colonel fabien,608721.2,5,16877281.0,0.036067,6090.315789,7.43,3.17,0.16,...,45.10,41.33,34.81,22.89,15.25,12.20,10.86,1218.063158,609.031579,70635.936842
2,118,menilmontant,569452.8,5,16877281.0,0.033741,5824.315789,13.64,6.06,0.25,...,37.85,37.05,33.65,24.10,18.15,17.53,17.79,1164.863158,582.431579,70635.936842
3,163,blanche,433514.0,5,16877281.0,0.025686,4236.421053,16.80,7.78,0.19,...,41.29,41.39,36.88,27.88,21.15,27.09,24.68,847.284211,423.642105,70635.936842
4,256,philippe auguste,252530.6,5,16877281.0,0.014963,2473.842105,5.48,2.24,0.16,...,49.12,42.83,32.67,21.55,13.32,12.08,9.82,494.768421,247.384211,70635.936842
5,268,monceau,229735.4,5,16877281.0,0.013612,2529.789474,2.91,1.19,0.15,...,62.67,65.78,47.89,22.54,11.59,7.52,5.00,505.957895,252.978947,70635.936842
6,33,place clichy,1032386.4,5,16877281.0,0.061170,10922.736842,13.84,5.13,0.13,...,41.68,40.48,38.04,26.06,20.67,21.62,21.26,2184.547368,1092.273684,70635.936842
7,38,la chapelle,971106.2,5,16877281.0,0.057539,13644.368421,4.04,1.16,0.00,...,49.12,47.07,40.78,31.32,21.51,13.47,8.41,2728.873684,1364.436842,70635.936842
8,52,jaures,811092.2,5,16877281.0,0.048058,11482.421053,11.44,4.55,0.18,...,44.07,41.76,36.84,24.98,20.83,16.90,15.14,2296.484211,1148.242105,70635.936842
9,249,avron,269104.2,5,16877281.0,0.015945,2578.736842,7.15,2.70,0.13,...,39.07,39.59,37.36,25.67,16.63,11.96,10.53,515.747368,257.873684,70635.936842


Tableau 'ligne numéro[3]' avec la nouvelle colonne 'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction' :


...

,Rang,Station,Trafic,nb_correspondance,fréquentation_totale_ligne,Part du trafic de la station par rapport à toute la ligne,moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station,00:00-01:00,01:00-02:00,02:00-03:00,...,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nombre_validations_totales_pour_la_ligne
0,242,rue du bac,284672.8,5,23051055.6,0.012350,2642.842105,7.98,2.21,0.04,...,57.77,57.82,48.87,23.13,15.46,12.09,11.93,528.568421,264.284211,105667.292763
1,288,rennes,172266.8,5,23051055.6,0.007473,1546.052632,4.45,1.57,0.08,...,58.20,56.48,44.05,23.63,10.44,10.24,8.72,309.210526,154.605263,105667.292763
2,4,montparnasse-bienvenue,4081444.8,5,23051055.6,0.177061,67112.684211,5.71,0.26,0.01,...,55.50,62.72,49.65,31.05,21.54,16.83,13.49,13422.536842,6711.268421,105667.292763
3,4,montparnasse-bienvenue,4081444.8,5,23051055.6,0.177061,67112.684211,8.99,1.58,0.08,...,40.52,44.32,41.17,32.13,24.22,21.47,19.70,13422.536842,6711.268421,105667.292763
4,27,madeleine,1066185.6,5,23051055.6,0.046253,12904.421053,4.97,1.52,0.11,...,65.73,71.49,60.81,23.76,11.64,11.65,10.10,2580.884211,1290.442105,105667.292763
5,88,sevres-babylone,678500.8,5,23051055.6,0.029435,6540.105263,4.71,1.28,0.07,...,59.22,66.54,57.85,35.27,12.25,9.71,8.96,1308.021053,654.010526,105667.292763
6,115,mairie d'issy,574827.6,5,23051055.6,0.024937,6616.526316,2.55,0.89,0.06,...,38.97,38.91,30.74,16.37,8.48,6.66,5.20,1323.305263,661.652632,105667.292763
7,188,notre-dame-de-lorette,378765.6,5,23051055.6,0.016432,3949.578947,10.62,4.42,0.04,...,44.18,49.69,39.57,25.65,17.51,18.02,18.33,789.915789,394.957895,105667.292763
8,93,porte de versailles,653631.4,5,23051055.6,0.028356,15023.947368,12.16,4.87,0.09,...,60.08,62.00,53.19,32.02,19.18,19.01,18.49,3004.789474,1502.394737,105667.292763
9,136,vaugirard,496577.2,5,23051055.6,0.021542,4768.736842,4.38,1.34,0.07,...,44.22,41.04,34.89,19.16,10.52,9.18,7.66,953.747368,476.873684,105667.292763


Tableau 'ligne numéro[13]' avec la nouvelle colonne 'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction' :


,Rang,Station,Trafic,nb_correspondance,fréquentation_totale_ligne,Part du trafic de la station par rapport à toute la ligne,moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station,00:00-01:00,01:00-02:00,02:00-03:00,...,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nombre_validations_totales_pour_la_ligne
0,74,saint-denis-universite,713998.0,5,16957848.6,0.042104,8049.368421,1.95,0.42,0.01,...,31.51,27.77,18.64,13.60,8.89,6.17,4.36,1609.873684,804.936842,37710.673684
1,122,les courtilles,560537.8,5,16957848.6,0.033055,8152.789474,2.39,0.58,0.02,...,35.24,31.76,23.70,16.18,10.77,7.34,4.88,1630.557895,815.278947,37710.673684
2,125,porte de saint-ouen,542127.6,5,16957848.6,0.031969,6262.368421,5.48,2.05,0.05,...,42.89,40.38,34.53,23.47,16.62,12.24,9.27,1252.473684,626.236842,37710.673684
3,29,porte de clichy,1055699.4,5,16957848.6,0.062254,12615.105263,9.95,3.26,0.27,...,92.43,88.85,71.86,46.94,32.62,24.38,18.66,2523.021053,1261.510526,37710.673684
4,33,place clichy,1032386.4,5,16957848.6,0.060880,10922.736842,13.84,5.13,0.13,...,41.68,40.48,38.04,26.06,20.67,21.62,21.26,2184.547368,1092.273684,37710.673684
5,87,porte de vanves,679071.6,5,16957848.6,0.040045,7711.315789,4.61,1.30,0.07,...,41.18,39.19,30.95,21.34,13.95,10.18,8.11,1542.263158,771.131579,37710.673684
6,162,pernety,434713.4,5,16957848.6,0.025635,4404.263158,5.99,1.92,0.09,...,34.66,34.29,31.53,21.05,13.67,12.71,10.96,880.852632,440.426316,37710.673684
7,230,carrefour pleyel,307105.6,5,16957848.6,0.018110,3933.789474,3.40,1.46,0.10,...,57.68,45.30,28.87,16.22,10.19,8.39,6.78,786.757895,393.378947,37710.673684
8,64,gabriel peri,758114.6,5,16957848.6,0.044706,9837.842105,3.24,0.79,0.02,...,38.88,38.49,29.48,18.40,11.34,7.23,5.40,1967.568421,983.784211,37710.673684
9,69,miromesnil,729451.0,5,16957848.6,0.043016,8603.789474,8.08,2.75,0.05,...,60.25,65.41,47.45,26.32,13.69,14.06,11.90,1720.757895,860.378947,37710.673684


Tableau 'ligne numéro[14]' avec la nouvelle colonne 'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction' :


,Rang,Station,Trafic,nb_correspondance,fréquentation_totale_ligne,Part du trafic de la station par rapport à toute la ligne,moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station,00:00-01:00,01:00-02:00,02:00-03:00,...,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nombre_validations_totales_pour_la_ligne
0,3,gare de lyon,5728095.0,5,15764240.2,0.363360,66037.947368,2.16,0.01,0.00,...,57.99,60.21,38.73,27.60,17.40,10.24,4.80,13207.589474,6603.794737,47415.801671
1,3,gare de lyon,5728095.0,5,15764240.2,0.363360,66037.947368,18.38,4.31,0.30,...,84.89,89.26,74.77,56.04,41.78,38.76,34.89,13207.589474,6603.794737,47415.801671
2,61,bercy,776842.4,5,15764240.2,0.049279,9023.947368,NaN,NaN,NaN,...,104.32,67.23,40.18,13.57,0.11,14.23,0.09,1804.789474,902.394737,47415.801671
3,61,bercy,776842.4,5,15764240.2,0.049279,9023.947368,10.59,2.39,0.22,...,43.36,44.68,36.17,35.22,24.11,25.58,36.50,1804.789474,902.394737,47415.801671
4,27,madeleine,1066185.6,5,15764240.2,0.067633,12904.421053,4.97,1.52,0.11,...,65.73,71.49,60.81,23.76,11.64,11.65,10.10,2580.884211,1290.442105,47415.801671
5,29,porte de clichy,1055699.4,5,15764240.2,0.066968,12615.105263,9.95,3.26,0.27,...,92.43,88.85,71.86,46.94,32.62,24.38,18.66,2523.021053,1261.510526,47415.801671
6,110,cour saint-emilion,597024.4,5,15764240.2,0.037872,3173.444444,11.21,1.93,0.12,...,61.54,60.91,45.62,29.16,25.69,15.63,12.76,634.688889,317.344444,47415.801671
7,6,bibliotheque,2220894.8,5,15764240.2,0.140882,6088.000000,6.46,1.35,0.06,...,54.02,56.83,44.83,26.99,18.84,11.58,8.76,1217.600000,608.800000,47415.801671
8,31,olympiades,1042919.0,5,15764240.2,0.066157,12997.142857,2.74,1.23,0.06,...,47.26,41.98,31.78,20.50,13.19,6.90,4.47,2599.428571,1299.714286,47415.801671
9,39,mairie de saint-ouen,966162.0,5,15764240.2,0.061288,12017.736842,7.75,2.71,0.19,...,93.80,89.63,63.45,40.80,24.30,16.35,13.66,2403.547368,1201.773684,47415.801671


### PROBLÈMES À RÉGLER : METTRE LES COLONNES DE CHAQUE TABLEAU DANS UN MEILLEUR ORDRE ET RENOMMER LES COLONNES POUR QUE CE SOIT + JOLI ### 

Trafic = le trafic annuel dans la station ; nb_correspondances = le nb de lignes dans la station ; fréquentation totale ligne = trafic annuel / nb_correspondances ; part du trafic de la station par rapport à toute la ligne = trafic / fréquentation_totale_ligne ; les titres de colonnes avec période_jo_2022 : moyenne hebdomadaire sur la période du 24-07-2022 au 11-08-2022

### TABLEAU A EXPLOITER CI DESSUS ###

In [35]:
# Créer le dictionnaire pour les nouvelles variables
tableaux_pour_variables = {}

for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Supprimer les colonnes spécifiées
    columns_to_drop = ['Rang', 'Trafic', 'nb_correspondance', 'fréquentation_totale_ligne', 
                       'Part du trafic de la station par rapport à toute la ligne',
                       'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station']

    # Créer un nouveau tableau sans les colonnes spécifiées
    new_table = current_table.drop(columns=columns_to_drop)

    # Ajouter le nouveau tableau dans le dictionnaire
    tableaux_pour_variables[f'ligne numéro[{k_str}]'] = new_table

    # Afficher le tableau mis à jour
    print(f"Tableau 'ligne numéro[{k_str}]' sans les colonnes spécifiées :")
    display(new_table)


Tableau 'ligne numéro[1]' sans les colonnes spécifiées :


,Station,00:00-01:00,01:00-02:00,02:00-03:00,03:00-04:00,04:00-05:00,05:00-06:00,06:00-07:00,07:00-08:00,08:00-09:00,...,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nombre_validations_totales_pour_la_ligne
0,gare de lyon,2.16,0.01,0.00,NaN,0.34,1.20,10.05,22.63,19.75,...,57.99,60.21,38.73,27.60,17.40,10.24,4.80,13207.589474,6603.794737,138177.887449
1,gare de lyon,18.38,4.31,0.30,0.10,0.62,8.89,19.53,38.87,63.64,...,84.89,89.26,74.77,56.04,41.78,38.76,34.89,13207.589474,6603.794737,138177.887449
2,saint-mande-tourelle,3.59,1.06,0.10,0.00,0.01,3.76,7.13,16.08,35.94,...,44.98,47.84,34.26,19.04,10.38,8.05,5.99,1745.536842,872.768421,138177.887449
3,louvre,14.09,4.62,0.10,0.00,0.02,1.16,1.90,4.38,10.59,...,59.32,63.31,49.76,28.05,19.06,19.83,20.34,874.894737,437.447368,138177.887449
4,porte de vincennes,3.90,1.27,0.10,0.00,0.00,5.48,12.00,22.18,35.34,...,40.51,38.45,30.06,18.82,12.32,9.06,6.66,2430.210526,1215.105263,138177.887449
5,porte maillot,2.64,0.07,0.02,0.00,0.03,1.79,5.27,13.49,20.72,...,68.87,69.45,51.21,31.13,17.93,11.92,7.54,1476.885830,738.442915,138177.887449
6,porte maillot,10.76,3.99,0.11,0.00,0.00,3.90,8.11,16.28,31.30,...,44.22,44.75,35.65,22.26,15.20,16.55,18.25,1476.885830,738.442915,138177.887449
7,chateau de vincennes,3.90,1.27,0.10,0.00,0.00,5.48,12.00,22.18,35.34,...,40.51,38.45,30.06,18.82,12.32,9.06,6.66,2430.210526,1215.105263,138177.887449
8,chatelet,30.17,10.60,0.27,0.04,0.36,7.19,5.32,8.82,15.78,...,114.31,122.78,105.60,70.85,51.84,47.85,44.60,17555.905263,8777.952632,138177.887449
9,pont de neuilly,3.28,1.22,0.13,0.03,0.02,2.50,5.23,13.45,28.72,...,46.88,51.98,39.18,21.86,11.94,9.25,6.68,2114.631579,1057.315789,138177.887449


Tableau 'ligne numéro[2]' sans les colonnes spécifiées :


,Station,00:00-01:00,01:00-02:00,02:00-03:00,03:00-04:00,04:00-05:00,05:00-06:00,06:00-07:00,07:00-08:00,08:00-09:00,...,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nombre_validations_totales_pour_la_ligne
0,belleville,7.73,3.56,0.12,0.00,0.00,4.47,7.21,14.32,24.10,...,41.62,37.49,32.01,21.30,14.72,13.35,11.97,3322.326316,1661.163158,70635.936842
1,colonel fabien,7.43,3.17,0.16,0.00,0.02,3.23,7.47,16.69,28.21,...,45.10,41.33,34.81,22.89,15.25,12.20,10.86,1218.063158,609.031579,70635.936842
2,menilmontant,13.64,6.06,0.25,NaN,0.00,5.44,9.84,17.15,27.60,...,37.85,37.05,33.65,24.10,18.15,17.53,17.79,1164.863158,582.431579,70635.936842
3,blanche,16.80,7.78,0.19,0.00,0.03,5.05,6.48,10.56,22.31,...,41.29,41.39,36.88,27.88,21.15,27.09,24.68,847.284211,423.642105,70635.936842
4,philippe auguste,5.48,2.24,0.16,0.00,0.02,3.26,5.99,14.16,28.02,...,49.12,42.83,32.67,21.55,13.32,12.08,9.82,494.768421,247.384211,70635.936842
5,monceau,2.91,1.19,0.15,0.00,0.17,0.92,1.87,6.95,17.83,...,62.67,65.78,47.89,22.54,11.59,7.52,5.00,505.957895,252.978947,70635.936842
6,place clichy,13.84,5.13,0.13,0.00,0.06,3.44,6.39,11.84,24.68,...,41.68,40.48,38.04,26.06,20.67,21.62,21.26,2184.547368,1092.273684,70635.936842
7,la chapelle,4.04,1.16,0.00,0.00,0.00,4.10,6.93,13.36,19.00,...,49.12,47.07,40.78,31.32,21.51,13.47,8.41,2728.873684,1364.436842,70635.936842
8,jaures,11.44,4.55,0.18,0.24,0.31,4.65,7.71,14.52,24.65,...,44.07,41.76,36.84,24.98,20.83,16.90,15.14,2296.484211,1148.242105,70635.936842
9,avron,7.15,2.70,0.13,0.00,0.01,2.76,5.69,14.08,29.15,...,39.07,39.59,37.36,25.67,16.63,11.96,10.53,515.747368,257.873684,70635.936842


Tableau 'ligne numéro[3]' sans les colonnes spécifiées :


...

,Station,00:00-01:00,01:00-02:00,02:00-03:00,03:00-04:00,04:00-05:00,05:00-06:00,06:00-07:00,07:00-08:00,08:00-09:00,...,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nombre_validations_totales_pour_la_ligne
0,rue du bac,7.98,2.21,0.04,0.00,0.00,0.92,2.17,5.96,13.43,...,57.77,57.82,48.87,23.13,15.46,12.09,11.93,528.568421,264.284211,105667.292763
1,rennes,4.45,1.57,0.08,0.00,0.09,1.01,1.74,6.23,14.50,...,58.20,56.48,44.05,23.63,10.44,10.24,8.72,309.210526,154.605263,105667.292763
2,montparnasse-bienvenue,5.71,0.26,0.01,0.04,0.24,3.15,13.07,24.17,21.87,...,55.50,62.72,49.65,31.05,21.54,16.83,13.49,13422.536842,6711.268421,105667.292763
3,montparnasse-bienvenue,8.99,1.58,0.08,0.00,0.00,2.24,4.74,16.88,31.99,...,40.52,44.32,41.17,32.13,24.22,21.47,19.70,13422.536842,6711.268421,105667.292763
4,madeleine,4.97,1.52,0.11,0.02,0.00,0.87,1.52,3.60,6.41,...,65.73,71.49,60.81,23.76,11.64,11.65,10.10,2580.884211,1290.442105,105667.292763
5,sevres-babylone,4.71,1.28,0.07,0.01,0.00,0.78,1.64,4.18,11.13,...,59.22,66.54,57.85,35.27,12.25,9.71,8.96,1308.021053,654.010526,105667.292763
6,mairie d'issy,2.55,0.89,0.06,0.00,0.05,4.60,9.49,24.97,45.36,...,38.97,38.91,30.74,16.37,8.48,6.66,5.20,1323.305263,661.652632,105667.292763
7,notre-dame-de-lorette,10.62,4.42,0.04,0.00,0.00,1.85,5.55,11.57,23.91,...,44.18,49.69,39.57,25.65,17.51,18.02,18.33,789.915789,394.957895,105667.292763
8,porte de versailles,12.16,4.87,0.09,0.01,0.00,1.58,2.33,4.42,9.79,...,60.08,62.00,53.19,32.02,19.18,19.01,18.49,3004.789474,1502.394737,105667.292763
9,vaugirard,4.38,1.34,0.07,0.00,0.00,1.95,6.72,15.98,33.64,...,44.22,41.04,34.89,19.16,10.52,9.18,7.66,953.747368,476.873684,105667.292763


Tableau 'ligne numéro[13]' sans les colonnes spécifiées :


,Station,00:00-01:00,01:00-02:00,02:00-03:00,03:00-04:00,04:00-05:00,05:00-06:00,06:00-07:00,07:00-08:00,08:00-09:00,...,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nombre_validations_totales_pour_la_ligne
0,saint-denis-universite,1.95,0.42,0.01,0.00,0.00,18.07,35.34,48.73,45.31,...,31.51,27.77,18.64,13.60,8.89,6.17,4.36,1609.873684,804.936842,37710.673684
1,les courtilles,2.39,0.58,0.02,0.00,0.00,11.63,20.74,38.10,47.37,...,35.24,31.76,23.70,16.18,10.77,7.34,4.88,1630.557895,815.278947,37710.673684
2,porte de saint-ouen,5.48,2.05,0.05,0.00,0.00,4.58,12.18,22.88,29.67,...,42.89,40.38,34.53,23.47,16.62,12.24,9.27,1252.473684,626.236842,37710.673684
3,porte de clichy,9.95,3.26,0.27,0.06,0.65,11.07,23.21,43.79,58.46,...,92.43,88.85,71.86,46.94,32.62,24.38,18.66,2523.021053,1261.510526,37710.673684
4,place clichy,13.84,5.13,0.13,0.00,0.06,3.44,6.39,11.84,24.68,...,41.68,40.48,38.04,26.06,20.67,21.62,21.26,2184.547368,1092.273684,37710.673684
5,porte de vanves,4.61,1.30,0.07,0.01,0.00,5.18,10.84,21.69,36.21,...,41.18,39.19,30.95,21.34,13.95,10.18,8.11,1542.263158,771.131579,37710.673684
6,pernety,5.99,1.92,0.09,0.01,0.00,3.69,8.90,21.05,42.71,...,34.66,34.29,31.53,21.05,13.67,12.71,10.96,880.852632,440.426316,37710.673684
7,carrefour pleyel,3.40,1.46,0.10,0.00,0.05,4.69,12.42,20.57,26.50,...,57.68,45.30,28.87,16.22,10.19,8.39,6.78,786.757895,393.378947,37710.673684
8,gabriel peri,3.24,0.79,0.02,0.00,0.00,7.05,15.01,29.43,43.35,...,38.88,38.49,29.48,18.40,11.34,7.23,5.40,1967.568421,983.784211,37710.673684
9,miromesnil,8.08,2.75,0.05,0.01,0.05,1.50,4.30,9.05,13.77,...,60.25,65.41,47.45,26.32,13.69,14.06,11.90,1720.757895,860.378947,37710.673684


Tableau 'ligne numéro[14]' sans les colonnes spécifiées :


,Station,00:00-01:00,01:00-02:00,02:00-03:00,03:00-04:00,04:00-05:00,05:00-06:00,06:00-07:00,07:00-08:00,08:00-09:00,...,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nombre_validations_totales_pour_la_ligne
0,gare de lyon,2.16,0.01,0.00,NaN,0.34,1.20,10.05,22.63,19.75,...,57.99,60.21,38.73,27.60,17.40,10.24,4.80,13207.589474,6603.794737,47415.801671
1,gare de lyon,18.38,4.31,0.30,0.10,0.62,8.89,19.53,38.87,63.64,...,84.89,89.26,74.77,56.04,41.78,38.76,34.89,13207.589474,6603.794737,47415.801671
2,bercy,NaN,NaN,NaN,NaN,NaN,6.98,10.94,2.74,30.49,...,104.32,67.23,40.18,13.57,0.11,14.23,0.09,1804.789474,902.394737,47415.801671
3,bercy,10.59,2.39,0.22,0.02,0.30,3.90,8.60,17.20,24.96,...,43.36,44.68,36.17,35.22,24.11,25.58,36.50,1804.789474,902.394737,47415.801671
4,madeleine,4.97,1.52,0.11,0.02,0.00,0.87,1.52,3.60,6.41,...,65.73,71.49,60.81,23.76,11.64,11.65,10.10,2580.884211,1290.442105,47415.801671
5,porte de clichy,9.95,3.26,0.27,0.06,0.65,11.07,23.21,43.79,58.46,...,92.43,88.85,71.86,46.94,32.62,24.38,18.66,2523.021053,1261.510526,47415.801671
6,cour saint-emilion,11.21,1.93,0.12,0.00,0.00,1.45,3.63,8.09,16.65,...,61.54,60.91,45.62,29.16,25.69,15.63,12.76,634.688889,317.344444,47415.801671
7,bibliotheque,6.46,1.35,0.06,0.00,0.10,5.02,9.13,13.29,19.76,...,54.02,56.83,44.83,26.99,18.84,11.58,8.76,1217.600000,608.800000,47415.801671
8,olympiades,2.74,1.23,0.06,0.00,0.01,6.71,10.26,17.73,32.47,...,47.26,41.98,31.78,20.50,13.19,6.90,4.47,2599.428571,1299.714286,47415.801671
9,mairie de saint-ouen,7.75,2.71,0.19,0.01,0.01,11.46,26.18,51.14,66.81,...,93.80,89.63,63.45,40.80,24.30,16.35,13.66,2403.547368,1201.773684,47415.801671


In [36]:
def affluence_par_heure(horaire, station, line):
    # Obtenir le tableau correspondant à la ligne
    line_table = tableaux_pour_variables[f'ligne numéro[{line}]']

    # Extraire l'heure de l'horaire entré par l'utilisateur
    hour, minute = map(int, horaire.split(':'))

    # Calculer l'indice de la colonne correspondant à la plage horaire donnée
    time_range_index = (hour * 60 + minute) // 60

    # Vérifier si l'indice est dans les limites du tableau
    if time_range_index < 0 or time_range_index >= len(line_table.columns):
        raise ValueError(f"Invalid time: {horaire}")

    # Filtrer le tableau pour la station donnée
    station_row = line_table[line_table['Station'] == station]

    if station_row.empty:
        raise ValueError(f"La station {station} n'est pas sur la ligne numéro {line}")

    # Récupérer la valeur dans la colonne de la plage horaire
    affluence_value = station_row.iloc[0, time_range_index]

    return affluence_value

# Exemple d'utilisation
line_number = "1"
station_name = "bastille"
time = "9:00"

try:
    affluence_value = affluence_par_heure(time, station_name, line_number)
    print(f"L'affluence à la station {station_name}, sur la ligne numéro {line_number}, à {time} est de : {affluence_value} personnes sur une heure")
except ValueError as e:
    print(e)


L'affluence à la station bastille, sur la ligne numéro 1, à 9:00 est de : 15.09 personnes sur une heure
